In [1]:
import requests
import json
from bs4 import BeautifulSoup
from csv import writer

client_id = 'client_80e72e5511155fdde9a18e467f829c15'
client_secret = 'secret_4e3af2f4c36e901b8f7c0afbda475f32'
scopes = ['api_listings_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
url_endpoint = 'https://api.domain.com.au/v1/listings/residential/_search'

response = requests.post(auth_url, data = {
                        'client_id':client_id,
                        'client_secret':client_secret,
                        'grant_type':'client_credentials',
                        'scope':scopes,
                        'Content-Type':'text/json'
                        })
json_res = response.json()
access_token=json_res['access_token']
print(access_token)
auth = {'Authorization':'Bearer ' + access_token}
url = url_endpoint

# Write column headers
headers = ['street_adress', 'suburb', 'postcode', 'area', 'property_type', 'bedrooms', 'bathrooms', 'carspaces', 'price', 'time', 'latitude', 'longitude']
output_csv = open('lst1.csv', 'w')
# create the csv writer object
csv_writer = writer(output_csv)
# first write the headers
csv_writer.writerow(headers)

output_csv.close()
content = []
#areas = ["Melbourne City Council - Greater Area",
#                  "Port Philip City Council - Greater Area",
#                  "Stonnington City Council - Greater Area",
#                  "Yarra City Council - Greater Area"]

# set how many pages we want to collect
for page in range(1,6):
    post_fields = {
          "listingType":"Rent",
          "pageNumber":page,
          "pageSize": 200,
          #"pageNumber": 2,
          "propertyTypes":"",
          "minBedrooms":1,
          "minBathrooms":1,
          #"maxPrice": 400,
          "sort": {
              "sortKey": "Default"
          },
          
          "locations":[
            {
              "state":"VIC",
              "region":"",
              #"area":"Melbourne City Council - Greater Area",
              #"area":"",
              "suburb":"",
              "postCode":"",
              "includeSurroundingSuburbs":False
            }
          ]
    }

    # Request content
    res1 = requests.post(url,headers=auth,json=post_fields)
    print(res1)
    content.append(json.loads(res1.text))
    print(f"Completed Page {len(content)}, this page has {len(content[len(content)-1])} data instances")   


5902a6cac030eed00f5da2a969d422e1
<Response [429]>
Completed Page 1, this page has 3 data instances
<Response [429]>
Completed Page 2, this page has 3 data instances
<Response [429]>
Completed Page 3, this page has 3 data instances
<Response [429]>
Completed Page 4, this page has 3 data instances
<Response [429]>
Completed Page 5, this page has 3 data instances


In [3]:
content[0]
# success content[0] = [{}, {}, {}, ... {}]
# failure content[0] = {'type':..., 'title': 'Quota Exceeded'}
# failure content[0] = {'errors': {'searchParameters.Page': ['Cannot page beyond 1000 records.']}, 'message': 'The request is invalid.'}

{'type': 'https://developer.domain.com.au/docs/latest/conventions/rate-limiting',
 'title': 'Quota Exceeded',
 'detail': 'Exceeded rate limit for current period'}

In [12]:
for each in content:
    # Get request for each property ID and extract attributes
    for i in each:
        #get details
        r = i['listing']['propertyDetails']
        street_address=r['displayableAddress']
        suburb=r['suburb']
        postcode=r['postcode']
        area=r['area']
        latitude = r['latitude']
        longitude = r['longitude']
        property_type=r['propertyType']
        bathrooms=r['bathrooms']
        bedrooms=r['bedrooms']
        
        #print(i['listing']['inspectionSchedule']['times'])
        if(i['listing']['dateListed']):
            time = i['listing']['dateListed']
        else:
            time = 0

        if(str(i).find('carspaces'))>0:
            carspaces=i['listing']['propertyDetails']['carspaces']
        else:
            carspaces=0
        
        price=i['listing']['priceDetails']['displayPrice'] #[1:4]

        info = [street_address, suburb, postcode, area, property_type, bedrooms, bathrooms, carspaces, price, time, latitude, longitude]
        
        # append the remaining rows one by one
        with open('lst1.csv', 'a', newline='') as g:
            thewriter = writer(g)        
            thewriter.writerow(info)



#### Visualise Property Schema (Do not run this)

In [4]:
"""
[
  {
    "type": "Project",
    "listings": [
      {
        "listingType": "NewHomes",
        "id": 2017835505,
        "projectId": 5437,
        "advertiser": {
          "type": "Agency",
          "id": 32584,
          "name": "Roche Group",
          "logoUrl": "https://images.domain.com.au/img/Agencys/32584/logo_32584.jpeg",
          "preferredColourHex": "#1d2c44",
          "bannerUrl": "https://images.domain.com.au/img/Agencys/32584/banner_32584.jpeg",
          "contacts": [
            {
              "name": "Cameron Grove Sales",
              "photoUrl": "https://images.domain.com.au/img/32584/contact_1637430.png?mod=220821-125111"
            }
          ]
        },
        "priceDetails": {
          "displayPrice": "RESERVED"
        },
        "media": [
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_1_1_220726_010938-w2880-h2160"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_2_1_220725_035031-w2048-h1536"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_3_1_220725_035031-w1536-h2048"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_4_1_220725_035031-w1536-h2048"
          },
          {
            "category": "Image",
            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_5_1_220725_035031-w2048-h1536"
          }
        ],
        "propertyDetails": {
          "state": "NSW",
          "features": [
            "AirConditioning",
            "Ensuite",
            "Gas",
            "Bath",
            "Dishwasher",
            "Study",
            "RainwaterStorageTank"
          ],
          "propertyType": "House",
          "allPropertyTypes": [
            "House"
          ],
          "bathrooms": 2,
          "bedrooms": 4,
          "carspaces": 2,
          "unitNumber": "",
          "streetNumber": "37",
          "street": "Estelville Circuit",
          "area": "Lake Macquarie West",
          "region": "Hunter Region",
          "suburb": "CAMERON PARK",
          "postcode": "2285",
          "displayableAddress": "37 Estelville Circuit, Cameron Park",
          "latitude": -32.91141,
          "longitude": 151.59761,
          "landArea": 586,
          "buildingArea": 265,
          "isRural": false,
          "isNew": true,
          "tags": [
            "newdevelopment"
          ]
        },
        "headline": "Cameron Park - Home under construction, amazing views!",
        "summaryDescription": "<b></b><br />Don't miss out on this one of a kind opportunity, this home is currently under construction, you will  see the interior of the home this weekend, Landscaping and Fencing are about to commence. Hope to see you here this weekend.\r\n\r\nTucked a...",
        "hasFloorplan": true,
        "hasVideo": false,
        "labels": [
          "Under offer",
          "Updated",
          "New homes"
        ],
        "dateListed": "2022-05-28T20:15:55",
        "inspectionSchedule": {
          "byAppointment": true,
          "recurring": false,
          "times": []
        },
        "listingSlug": "37-estelville-circuit-cameron-park-nsw-2285-2017835505"
      }
    ],
    "project": {
      "state": "NSW",
      "id": 5437,
      "name": "Cameron Grove Estate",
      "bannerUrl": "https://images.domain.com.au/img/Agencys/devproject/banner_5437_220726_011050",
      "preferredColorHex": "#419a45",
      "logoUrl": "https://images.domain.com.au/img/Agencys/devproject/logo_5437_220726_011050",
      "labels": [
        "New"
      ],
      "displayableAddress": "28 28 BREAKWELL ROAD, CAMERON PARK, NSW 2285",
      "suburb": "CAMERON PARK",
      "features": [
        ""
      ],
      "media": [
        {
          "category": "Image",
          "url": "https://bucket-api.domain.com.au/v1/bucket/image/af4a92fb-1686-479d-a6f7-13e7fc1425b8-w1500-h966"
        }
      ],
      "projectSlug": "project/5437/cameron-grove-estate-cameron-park-nsw"
    }
  }
]

"""

'\n[\n  {\n    "type": "Project",\n    "listings": [\n      {\n        "listingType": "NewHomes",\n        "id": 2017835505,\n        "projectId": 5437,\n        "advertiser": {\n          "type": "Agency",\n          "id": 32584,\n          "name": "Roche Group",\n          "logoUrl": "https://images.domain.com.au/img/Agencys/32584/logo_32584.jpeg",\n          "preferredColourHex": "#1d2c44",\n          "bannerUrl": "https://images.domain.com.au/img/Agencys/32584/banner_32584.jpeg",\n          "contacts": [\n            {\n              "name": "Cameron Grove Sales",\n              "photoUrl": "https://images.domain.com.au/img/32584/contact_1637430.png?mod=220821-125111"\n            }\n          ]\n        },\n        "priceDetails": {\n          "displayPrice": "RESERVED"\n        },\n        "media": [\n          {\n            "category": "Image",\n            "url": "https://bucket-api.domain.com.au/v1/bucket/image/2017835505_1_1_220726_010938-w2880-h2160"\n          },\n        